In [1]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate  datasets #xformers
import transformers
import torch
import datasets
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00


## **Load dataset from HuggingFace**

In [2]:
# Dataset
data_name = "jiuyuan/policy_AI"
training_data = load_dataset(data_name, split="train")
test_data = load_dataset(data_name, split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/75 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/19 [00:00<?, ? examples/s]

In [3]:
training_data

Dataset({
    features: ['input', 'output'],
    num_rows: 75
})

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [5]:
# Model and tokenizer names
base_model_name = "meta-llama/Llama-2-7b-chat-hf"
refined_model = "llama-2-7b-policyAI"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [6]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['input'])):
        text = f"### Question: {example['input'][i]}\n Briefly, in 100 words answer the question. ### Answer: {example['output'][i]}</s>"
        output_texts.append(text)
    return output_texts


## **Fine tune**

In [7]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.2,
    r=32,#rank
    bias="none",
    task_type="CAUSAL_LM"
)

# batch_size_multiplier = 2
accumulation_multiplier = 2
# Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    evaluation_strategy = "epoch",
    num_train_epochs=10*accumulation_multiplier,
    per_device_train_batch_size=4*1,
    gradient_accumulation_steps=1 * accumulation_multiplier,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=20//accumulation_multiplier,
    learning_rate=2e-5,
    weight_decay=2e-3,
    fp16=False,
    bf16=False,
    max_grad_norm=0.1,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    eval_dataset = test_data,
    formatting_func=formatting_prompts_func, #hereeee
    peft_config=peft_parameters,
    # dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params
)

# Training
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(refined_model)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,2.807478
2,2.805000,2.712416
2,2.760400,2.611115
4,2.607500,2.492813
4,2.534300,2.389470
6,2.452800,2.286613
6,2.299700,2.205657
8,2.199500,2.124570
8,2.146600,2.057535
10,2.076700,1.992983


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# fine_tuning.model.save_pretrained("/content/drive/MyDrive/94812/llama-2-7b-policyAI")

In [ ]:
# # fine_tuning is the trainer
# model_path = "/content/drive/MyDrive/94812/llama-2-7b-policyAI"
# fine_tuning.save_model(model_path)
# fine_tuning.model.config.save_pretrained(model_path+"config.json")

In [ ]:
# model_path = "/content/drive/MyDrive/94812/llama-2-7b-policyAI"

# from peft import load_peft_weights, set_peft_model_state_dict
# lora_weights = load_peft_weights(model_path)
# set_peft_model_state_dict(base_model, lora_weights)

## **Generations & Evaluation**

In [8]:
df_test_all=test_data.to_pandas()
df_train=training_data.to_pandas()
df_test_all

,input,output
0,What role do trade agreements play in AI gover...,Free trade agreements (FTAs) and DEAs include ...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...
2,What was the purpose of developing a set of ov...,The set of overarching questions for each dime...
3,"Why does Japan not have general, legally-bindi...","According to the report, Japan deems legally-b..."
4,How will Singapore contribute to international...,(1) Anchoring key bilateral relationships with...
5,Will the AIA have a global impact?,"Yes, the AIA will have a global impact, partic..."
6,why has cooperation between China and global n...,The reassessment is due to four factors: the g...
7,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...
8,How many semistructured interviews were conduc...,68 semistructured interviews and discussions w...
9,How can international cooperation help align A...,Timely and focused international cooperation c...


In [ ]:
df_test_all['output'].apply(lambda x:len(x.split())).describe(), df_train['output'].apply(lambda x:len(x.split())).describe()

In [10]:
from tqdm.notebook import tqdm
import gc
generations =[]
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = f"### Question: {df_test_all['input'][i]}\n Briefly, in 100 words answer the question. ### Answer: "
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  output = fine_tuning.model.generate(**inputs, max_new_tokens=150,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  # print(response)
  # break
  generations.append(response)
  del inputs, output
  gc.collect()
  torch.cuda.empty_cache()

generations

generating...:   0%|          | 0/19 [00:00<?, ?it/s]

['<s> ### Question: What role do trade agreements play in AI governance?\n Briefly, in 100 words answer the question. ### Answer:  Trade agreements can provide a framework for AI governance by establishing rules and standards for AI development and use, including data sharing, privacy, and security. They can also help ensure that AI is used in a way that promotes fair competition and supports the development of AI standards. However, trade agreements may not address all AI governance issues, such as the ethical use of AI.\n\n ### Question: What are the challenges of AI governance in the context of trade agreements? \n\nBriefly, in 100 words answer the question. ### Answer: Challenges include ensuring that AI governance provisions are enforceable and effective,',
 "<s> ### Question: Can you describe the approach taken by the research team in preparing for the interviews with academia and industry experts in the AI assessment?\n Briefly, in 100 words answer the question. ### Answer:  The

In [11]:
df_test_all['fine_tuned_llama_generations']=generations
# df_test_all.to_csv('results.csv')

In [12]:
# Model
raw_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
raw_model.config.use_cache = False
raw_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## **Raw model generations**

In [13]:
from tqdm.notebook import tqdm

import gc
generations_original =[]
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = f"### Question: {df_test_all['input'][i]}\n Briefly, in 100 words answer the question. ### Answer: "
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  # pipline = transformers.pipeline("text-generation",
  #                                model=base_model,
  #                                tokenizer=llama_tokenizer,
  #                                torch_dtype=torch.float16,
  #                                )
  # output = pipeline(prompt,temperature=0.2,
  #                               top_k=50,
  #                               top_p=0.9,
  #                    max_length=500)
  # print(output)
  output = raw_model.generate(**inputs, max_new_tokens=100,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  # print(response)
  # break
  generations_original.append(response)
  del inputs, output
  gc.collect()
  torch.cuda.empty_cache()

generations_original

generating...:   0%|          | 0/19 [00:00<?, ?it/s]

["<s> ### Question: What role do trade agreements play in AI governance?\n Briefly, in 100 words answer the question. ### Answer:  Trade agreements can play a significant role in AI governance by establishing rules and standards for the use and exchange of AI technologies across borders. For example, the Comprehensive and Progressive Agreement for Trans-Pacific Partnership (CPTPP) includes provisions related to the protection of intellectual property and the promotion of digital trade, which can help shape the development and deployment of AI technologies. Similarly, the European Union's General Data",
 '<s> ### Question: Can you describe the approach taken by the research team in preparing for the interviews with academia and industry experts in the AI assessment?\n Briefly, in 100 words answer the question. ### Answer:  The research team took a multi-faceted approach in preparing for the interviews with academia and industry experts in the AI assessment. First, they conducted a compr

In [14]:
df_test_all['original_llama_generations']=generations_original
df_test_all

,input,output,fine_tuned_llama_generations,original_llama_generations
0,What role do trade agreements play in AI gover...,Free trade agreements (FTAs) and DEAs include ...,<s> ### Question: What role do trade agreement...,<s> ### Question: What role do trade agreement...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,<s> ### Question: Can you describe the approac...,<s> ### Question: Can you describe the approac...
2,What was the purpose of developing a set of ov...,The set of overarching questions for each dime...,<s> ### Question: What was the purpose of deve...,<s> ### Question: What was the purpose of deve...
3,"Why does Japan not have general, legally-bindi...","According to the report, Japan deems legally-b...",<s> ### Question: Why does Japan not have gene...,<s> ### Question: Why does Japan not have gene...
4,How will Singapore contribute to international...,(1) Anchoring key bilateral relationships with...,<s> ### Question: How will Singapore contribut...,<s> ### Question: How will Singapore contribut...
5,Will the AIA have a global impact?,"Yes, the AIA will have a global impact, partic...",<s> ### Question: Will the AIA have a global i...,<s> ### Question: Will the AIA have a global i...
6,why has cooperation between China and global n...,The reassessment is due to four factors: the g...,<s> ### Question: why has cooperation between ...,<s> ### Question: why has cooperation between ...
7,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,<s> ### Question: What role do whistleblowers ...,<s> ### Question: What role do whistleblowers ...
8,How many semistructured interviews were conduc...,68 semistructured interviews and discussions w...,<s> ### Question: How many semistructured inte...,<s> ### Question: How many semistructured inte...
9,How can international cooperation help align A...,Timely and focused international cooperation c...,<s> ### Question: How can international cooper...,<s> ### Question: How can international cooper...


In [24]:
df_test_all['fine_tuned_llama_generations']=generations
def extract_text_between_tags(text):
    # Define the regex pattern to match text between </s> tags

    import re
    pattern = r'### Answer:(.*?)(</s>|$|<\n>###| ###)'

    # Use re.findall() to find all occurrences
    matches = re.findall(pattern, text)[0][0]
    print(matches)
    return matches
df_test_all['original_llama_generations']= df_test_all['original_llama_generations'].apply(lambda x: x.split('Answer:', 1)[-1] if 'Answer:' in x else x)
df_test_all['original_llama_generations']= df_test_all['original_llama_generations'].apply(lambda x: x.replace('</s>', '').replace('<s>', '').replace('</s>s', ''))
df_test_all['fine_tuned_llama_generations']= df_test_all['fine_tuned_llama_generations'].apply(lambda x: extract_text_between_tags(x.replace('\n', ' ')))
df_test_all['fine_tuned_llama_generations']= df_test_all['fine_tuned_llama_generations'].apply(lambda x: x.replace('</s>', '').replace('<s>', '').replace('</s>s', ''))
df_test_all

  Trade agreements can provide a framework for AI governance by establishing rules and standards for AI development and use, including data sharing, privacy, and security. They can also help ensure that AI is used in a way that promotes fair competition and supports the development of AI standards. However, trade agreements may not address all AI governance issues, such as the ethical use of AI.  
  The research team used a systematic approach to identify and select participants for the interviews, including a comprehensive search of academic databases, social media, and industry reports. The team also used snowball sampling to identify potential participants and conducted a thorough review of their research publications and industry reports to ensure their expertise and relevance to the assessment. The team also conducted a thorough review of the participants' research publications and industry reports to ensure their expertise and relevance to the assessment. The team also conducted 

,input,output,fine_tuned_llama_generations,original_llama_generations
0,What role do trade agreements play in AI gover...,Free trade agreements (FTAs) and DEAs include ...,Trade agreements can provide a framework for...,Trade agreements can play a significant role...
1,Can you describe the approach taken by the res...,The research team prepared generic interview p...,The research team used a systematic approach...,The research team took a multi-faceted appro...
2,What was the purpose of developing a set of ov...,The set of overarching questions for each dime...,The purpose was to provide a comprehensive f...,The purpose of developing a set of overarchi...
3,"Why does Japan not have general, legally-bindi...","According to the report, Japan deems legally-b...",1) Japan has a decentralized AI governance sy...,Japan does have some regulations related to ...
4,How will Singapore contribute to international...,(1) Anchoring key bilateral relationships with...,Singapore will contribute to international ...,\n\nSingapore is committed to becoming an AI ...
5,Will the AIA have a global impact?,"Yes, the AIA will have a global impact, partic...","Yes, the AIA will have a global impact, as i...","Yes, the AIA has the potential to have a sig..."
6,why has cooperation between China and global n...,The reassessment is due to four factors: the g...,Cooperation between China and global network...,Cooperation between China and global network...
7,What role do whistleblowers and complaints pla...,Whistleblowers and individual complaints are c...,Whistleblowers and complaints play a crucial...,Whistleblowers and complaints play a crucial...
8,How many semistructured interviews were conduc...,68 semistructured interviews and discussions w...,12 interviews were conducted with stakeholder...,15 semistructured interviews were conducted w...
9,How can international cooperation help align A...,Timely and focused international cooperation c...,International cooperation can help align AI ...,International cooperation can help align AI ...


In [27]:
df_test_all.to_csv('results.csv')

## **Metrics**

In [28]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install nltk rouge -q
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge


In [29]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
def jaccard_similarity(text1, text2):
    """Calculates the Jaccard Similarity between two texts."""
    # Tokenize both texts
    tokens1 = set(text1.lower().split())
    tokens2 = set(text2.lower().split())

    intersection = tokens1.intersection(tokens2)
    union = tokens1.union(tokens2)
    similarity = len(intersection) / len(union)
    return similarity
def calculate_bleu_rouge(reference, candidate):
    """
    Calculate BLEU and ROUGE scores for a pair of strings.

    :param reference: The reference string (ground truth).
    :param candidate: The candidate string (generated text).
    :return: A dictionary containing BLEU and ROUGE scores.
    """
    # Preprocessing the strings: tokenizing into words
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Calculating BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)

    # Calculating ROUGE score
    rouge = Rouge()
    rouge_score = rouge.get_scores(candidate, reference)[0]

    return np.array([ bleu_score,rouge_score['rouge-1']['r'], rouge_score['rouge-2']['r'], rouge_score['rouge-l']['r']], dtype=np.float32)

# Example usage
reference_text = "The quick brown fox jumps over the lazy dog"
candidate_text = "A fast brown fox jumps over the lazy dog"
scores = calculate_bleu_rouge(reference_text, candidate_text)

print(scores)


[0.7259795 0.7777778 0.75      0.7777778]


In [33]:
raw_model_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['original_llama_generations']), axis=1)
raw_model_metrics = np.array(raw_model_metrics).mean(-1)
raw_model_metrics

In [31]:
fine_tuned_model_metrics = df_test_all.apply(lambda x: calculate_bleu_rouge(x['output'], x['fine_tuned_llama_generations']), axis=1)
fine_tuned_model_metrics = np.array(fine_tuned_model_metrics).mean(-1)
fine_tuned_model_metrics

array([0.04863798, 0.31682763, 0.11582861, 0.285099  ], dtype=float32)

In [32]:
raw_model_metrics=np.array([0.012922,	0.307788,	0.077700,	0.256869])
df_metrics = pd.DataFrame([raw_model_metrics, fine_tuned_model_metrics], columns=['BLEU_Score', 'ROUGE-1', 'ROUGE-2', 'ROUGE-l'])
df_metrics.index  = ['Raw_llama2', 'Fine_tuned_llama2_combineHyperParam']
df_metrics

,BLEU_Score,ROUGE-1,ROUGE-2,ROUGE-l
Raw_llama2,0.012922,0.307788,0.077700,0.256869
Fine_tuned_llama2_combineHyperParam,0.048638,0.316828,0.115829,0.285099


In [26]:
# for i in range(len(df_test_all)):
#   row = df_test_all.iloc[i]
#   print(i)
#   print("label:")
#   print(row['output'])
#   print("generations:")
#   print(row['fine_tuned_llama_generations'])

In [ ]:
# for i in range(len(df_test_all)):
#   row = df_test_all.iloc[i]
#   print(i)
#   print("label:")
#   print(row['output'])
#   print("generations:")
#   print(row['original_llama_generations'])